In [1]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.fillna(-1)
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:19<00:00, 12.50it/s]


Maximum younden,specificity, sensitivity, threshold  (0.388885945967972, 0.4662055335968379, 0.9226804123711341, 0.9179179179179179)


100%|██████████| 10/10 [00:00<00:00, 104.17it/s]

0.7806466729147141


In [2]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.fillna(-1)
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:18<00:00, 12.82it/s]


Maximum younden,specificity, sensitivity, threshold  (0.3811682490526058, 0.4492094861660079, 0.931958762886598, 0.5745745745745746)


100%|██████████| 10/10 [00:00<00:00, 114.91it/s]

0.7774550751803104


In [3]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.dropna()
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:07<00:00, 14.75it/s]


Maximum younden,specificity, sensitivity, threshold  (0.6080376280066683, 0.6874493927125506, 0.9205882352941177, 0.26626626626626626)


100%|██████████| 10/10 [00:00<00:00, 119.03it/s]

0.8732555370326269


In [4]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.dropna()
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:14<00:00, 13.43it/s]


Maximum younden,specificity, sensitivity, threshold  (0.608359133126935, 0.6789473684210526, 0.9294117647058824, 0.4724724724724725)


100%|██████████| 10/10 [00:00<00:00, 121.96it/s]

0.866432483924744


In [5]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.dropna()
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)) stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-5-843ea1d860ac>, line 62)

In [6]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.dropna()
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:08<00:00, 14.71it/s]


Maximum younden,specificity, sensitivity, threshold  (0.6666666666666667, 0.7, 0.9666666666666666, 0.9519519519519519)


100%|██████████| 10/10 [00:00<00:00, 85.48it/s]

0.8642176708740177


In [7]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.dropna()
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:12<00:00, 13.74it/s]


Maximum younden,specificity, sensitivity, threshold  (0.6373333333333333, 0.704, 0.9333333333333333, 0.5875875875875876)


100%|██████████| 10/10 [00:00<00:00, 149.25it/s]

0.7519999999999999


In [8]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.dropna()
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:01<00:00, 16.32it/s]


Maximum younden,specificity, sensitivity, threshold  (0.6520000000000001, 0.752, 0.9, 0.24024024024024024)


100%|██████████| 10/10 [00:00<00:00, 142.86it/s]

0.804


In [9]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.fillna(-1)
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:12<00:00, 13.79it/s]


Maximum younden,specificity, sensitivity, threshold  (0.39799999999999996, 0.47800000000000004, 0.9200000000000002, 0.9009009009009009)


100%|██████████| 10/10 [00:00<00:00, 135.13it/s]

0.7515000000000001


In [10]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.fillna(-1)
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = max(scores, key=lambda score: score[0])
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:13<00:00, 13.54it/s]


Maximum younden,specificity, sensitivity, threshold  (0.39, 0.47000000000000003, 0.9200000000000002, 0.8018018018018018)


100%|██████████| 10/10 [00:00<00:00, 124.96it/s]

0.708


In [11]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.fillna(-1)
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = [max(scores, key=lambda score: score[0]), min(scores, key=lambda score: score[0])]
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [00:59<00:00, 16.91it/s]


Maximum younden,specificity, sensitivity, threshold  [(0.378, 0.46799999999999997, 0.9099999999999999, 0.8228228228228228), (0.052000000000000005, 0.262, 0.7899999999999999, 0.23523523523523523)]


100%|██████████| 10/10 [00:00<00:00, 142.86it/s]

0.7536


In [12]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.fillna(-1)
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = [max(scores, key=lambda score: score[0]), mean(scores, key=lambda score: score[0])]
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:05<00:00, 15.22it/s]


NameError: name 'mean' is not defined

In [13]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.fillna(-1)
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = [max(scores, key=lambda score: score[0]), np.mean(scores, key=lambda score: score[0])]
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:00<00:00, 16.52it/s]


TypeError: _mean_dispatcher() got an unexpected keyword argument 'key'

In [14]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
data = data.fillna(-1)
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = [max(scores, key=lambda score: score[0]), np.mean(scores,axis=0)]
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

100%|██████████| 1000/1000 [01:10<00:00, 14.24it/s]


Maximum younden,specificity, sensitivity, threshold  [(0.376, 0.506, 0.8700000000000001, 0.92992992992993), array([0.220954, 0.366904, 0.85405 , 0.5     ])]


100%|██████████| 10/10 [00:00<00:00, 131.58it/s]

0.7846


In [15]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
#print(pd.isna(data).sum())
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = [max(scores, key=lambda score: score[0]), np.mean(scores,axis=0)]
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [16]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
print(data.mean)

<bound method NDFrame._add_numeric_operations.<locals>.mean of       LeicGender  LeicRace  raeducl  mstat  shlt  hlthlm  mobilb  lgmusa  \
0              0         0      2.0      3     2       0       0       0   
1              1         0      2.0      1     2       0       0       0   
2              0         0      3.0      5     3       0       0       1   
3              1         0      3.0      1     2       1       0       0   
4              0         0      2.0      1     2       0       1       0   
...          ...       ...      ...    ...   ...     ...     ...     ...   
2004           0         0      2.0      7     3       0       0       0   
2005           1         0      1.0      5     4       0       0       1   
2006           1         0      4.0      7     3       0       0       0   
2007           1         0      4.0      7     1       0       0       0   
2008           1         0      2.0      1     5       1       2       0   

      grossa  finea  ...

In [17]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
print(data.mean())

LeicGender         0.466401
LeicRace           0.018915
raeducl            2.016650
mstat              2.537083
shlt               2.657541
hlthlm             0.263813
mobilb             0.496267
lgmusa             0.833250
grossa             0.310602
finea              0.157790
LeicHBP            0.377800
LeicAge            0.921354
hearte             0.159283
psyche             0.099054
bmicat             3.194126
physActive         0.890991
drinkd_e           2.621970
smoken             0.126059
itot           27141.720466
cfoodo1m          53.909774
jphysa             3.693153
estwt             79.092185
wstva             97.015530
chol               5.806188
hdl                1.577166
ldl                3.462886
trig               1.685767
sys1             138.937500
dias3             87.895792
fglu               5.198584
hba1c             17.556384
hemda              0.300647
eatVegFru          0.982081
everHighGlu        0.659034
rYdiabe            0.160279
dtype: float64


In [18]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
print(data.describe())

        LeicGender     LeicRace      raeducl        mstat         shlt  \
count  2009.000000  2009.000000  1982.000000  2009.000000  2009.000000   
mean      0.466401     0.018915     2.016650     2.537083     2.657541   
std       0.498994     0.136258     1.066788     2.449234     1.078675   
min       0.000000     0.000000     0.000000     1.000000     1.000000   
25%       0.000000     0.000000     1.000000     1.000000     2.000000   
50%       0.000000     0.000000     2.000000     1.000000     3.000000   
75%       1.000000     0.000000     3.000000     5.000000     3.000000   
max       1.000000     1.000000     4.000000     8.000000     5.000000   

            hlthlm       mobilb      lgmusa       grossa        finea  ...  \
count  2009.000000  2009.000000  2009.00000  2009.000000  2009.000000  ...   
mean      0.263813     0.496267     0.83325     0.310602     0.157790  ...   
std       0.443063     0.888231     1.18950     0.843945     0.447982  ...   
min       0.000000   

In [19]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
print(data.groupby(['rYdiabe']).describe())

        LeicGender                                              LeicRace  \
             count      mean       std  min  25%  50%  75%  max    count   
rYdiabe                                                                    
0           1687.0  0.452282  0.497865  0.0  0.0  0.0  1.0  1.0   1687.0   
1            322.0  0.540373  0.499143  0.0  0.0  1.0  1.0  1.0    322.0   

                   ... eatVegFru      everHighGlu                           \
             mean  ...       75%  max       count      mean       std  min   
rYdiabe            ...                                                       
0        0.016005  ...       1.0  1.0      1687.0  0.558388  0.789912  0.0   
1        0.034161  ...       1.0  1.0       322.0  1.186335  0.571438  0.0   

                             
         25%  50%  75%  max  
rYdiabe                      
0        0.0  0.0  1.0  2.0  
1        1.0  1.0  2.0  2.0  

[2 rows x 272 columns]


In [20]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
print(data.groupby(['rYdiabe']).describe())

        LeicGender                                              LeicRace  \
             count      mean       std  min  25%  50%  75%  max    count   
rYdiabe                                                                    
0           1687.0  0.452282  0.497865  0.0  0.0  0.0  1.0  1.0   1687.0   
1            322.0  0.540373  0.499143  0.0  0.0  1.0  1.0  1.0    322.0   

                   ... eatVegFru      everHighGlu                           \
             mean  ...       75%  max       count      mean       std  min   
rYdiabe            ...                                                       
0        0.016005  ...       1.0  1.0      1687.0  0.558388  0.789912  0.0   
1        0.034161  ...       1.0  1.0       322.0  1.186335  0.571438  0.0   

                             
         25%  50%  75%  max  
rYdiabe                      
0        0.0  0.0  1.0  2.0  
1        1.0  1.0  2.0  2.0  

[2 rows x 272 columns]


In [21]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
print(data.groupby(['rYdiabe']).describe())

        LeicGender                                              LeicRace  \
             count      mean       std  min  25%  50%  75%  max    count   
rYdiabe                                                                    
0           1687.0  0.452282  0.497865  0.0  0.0  0.0  1.0  1.0   1687.0   
1            322.0  0.540373  0.499143  0.0  0.0  1.0  1.0  1.0    322.0   

                   ... eatVegFru      everHighGlu                           \
             mean  ...       75%  max       count      mean       std  min   
rYdiabe            ...                                                       
0        0.016005  ...       1.0  1.0      1687.0  0.558388  0.789912  0.0   
1        0.034161  ...       1.0  1.0       322.0  1.186335  0.571438  0.0   

                             
         25%  50%  75%  max  
rYdiabe                      
0        0.0  0.0  1.0  2.0  
1        1.0  1.0  2.0  2.0  

[2 rows x 272 columns]


In [22]:
#print(pd.isna(data).sum())
X = data[all_labels[:-1]] # get the features
y= data[all_labels[len(all_labels)-1]]#data[all_labels[len(all_labels)-1]] # get the target class
#from mixed_naive_bayes import MixedNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
categorical_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,20,32,33,34]
clf = GaussianNB()
roc = list()
scores = list()
thresholds = np.linspace(0, 1, 1000)
for thr in tqdm.tqdm(thresholds):
   younden = list()
   for i in range(0,10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=round(0.03*len(X)),
       train_size=round(0.07*len(X)), stratify=y)
      clf.fit(X_train,y_train)
      y_pred =(clf.predict_proba(X_test)[:,1]>=0.003).astype(bool)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
      specificity = tn/(tn+fp)
      sensitivity = tp/(tp+fn)
      younden.append([specificity+sensitivity-1,specificity, sensitivity])
   scores.append((sum(you[0] for you in younden)/len(younden),
   sum(you[1] for you in younden)/ len(younden), # per threshold
   sum(you[2] for you in younden)/ len(younden), thr))
optimal = [max(scores, key=lambda score: score[0]), np.mean(scores,axis=0)]
print('Maximum younden,specificity, sensitivity, threshold ', optimal)

for i in tqdm.tqdm(range(0,10)):
   X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=round(0.03*len(X)), train_size=round(0.07*len(X)), stratify=y)
   clf.fit(X_train,y_train)
   y_pred =clf.predict_proba(X_test)[:,1]
   roc.append(roc_auc_score(y_test, y_pred))
print(np.mean(roc))
#clf = MixedNB(categorical_features=categorical_features)
#X.to_csv('NaiveBayes.csv',header=False, index=False)
#X = pd.read_csv('NaiveBayes.csv', header=None, index_col=False)
#print(pd.isna(X).sum())
#print(X.head(1371))
#clf.predict(X)
#print(X)
# Apply machine learning techniques

  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [23]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
print(data.groupby(['rYdiabe']).describe())

        LeicGender                                              LeicRace  \
             count      mean       std  min  25%  50%  75%  max    count   
rYdiabe                                                                    
0           1687.0  0.452282  0.497865  0.0  0.0  0.0  1.0  1.0   1687.0   
1            322.0  0.540373  0.499143  0.0  0.0  1.0  1.0  1.0    322.0   

                   ... eatVegFru      everHighGlu                           \
             mean  ...       75%  max       count      mean       std  min   
rYdiabe            ...                                                       
0        0.016005  ...       1.0  1.0      1687.0  0.558388  0.789912  0.0   
1        0.034161  ...       1.0  1.0       322.0  1.186335  0.571438  0.0   

                             
         25%  50%  75%  max  
rYdiabe                      
0        0.0  0.0  1.0  2.0  
1        1.0  1.0  2.0  2.0  

[2 rows x 272 columns]


In [24]:
from turtle import width
from arfftocsv import processing
import numpy as np
import pandas as pd
import tqdm

all_labels = ['LeicGender','LeicRace','raeducl','mstat','shlt','hlthlm',
'mobilb','lgmusa','grossa','finea','LeicHBP','LeicAge','hearte',
'psyche','bmicat','physActive','drinkd_e','smoken','itot','cfoodo1m',
'jphysa','estwt','wstva','chol','hdl','ldl','trig','sys1','dias3',
'fglu','hba1c','hemda','eatVegFru','everHighGlu','rYdiabe']
# this function deletes @ and empty lines so that produce a 

to_replace = {'LeicAge': ['50-59', '60-69', '>=70'], 'LeicGender': ['Female', 'Male'], 
'bmicat': ["'1.underweight less than 18.5'",
 "'2.normal weight from 18.5 to 24.9'", "'3.pre-obesity from 25 to 29.9'",
 "'4.obesity class 1 from 30 to 34.9'", "'5.obesity class 2 from 35 to 39.9'",
  "'6.obesity class 3 greater than 40'"],
'LeicRace': [0, 6], 'hemda': ["'Not applicable'", 'Yes', 'No'],
 'wstva':[],'LeicHBP': ['No', 'Yes'], 'rYdiabe': ['0.no', '1.yes'],
 'raeducl':["'2.upper secondary and vocational training'",'3.tertiary',
 "'1.less than secondary'",'.o:other',"'.h:missing HSE value'",".m:Missing"],
  'mstat':['3.partnered','1.married','5.divorced','7.widowed','4.separated',"'8.never married'"],
  'shlt':["'2.Very good'",'3.Good','1.Excellent','4.Fair','5.Poor'], 'hlthlm':['0.no','1.yes','.d:DK'],
  'hearte':['0.no','1.yes'],'psyche':['0.no','1.yes'], 'physActive':['Yes','No'], 'smoken':['0.No',
  '1.Yes',".m:Missing"],
   'jphysa': ["'1.Sedentary occupation'","'3.Physical work'","'2.Standing occupation'","'.w:not working'",
   "'4.Heavy manual work'",'.m:Missing'], 'everHighGlu':['No',"'Not applicable'",'Yes'],
   'eatVegFru':['Yes', 'No'],'mobilb':[],'lgmusa':[], 'grossa':[],'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':
   [], 'estwt':[], 'chol':[],'ldl':[], 'hdl':[], 'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
}

values = {'LeicAge': [0, 1, 2], 'LeicGender': [0, 1],
 'bmicat': [1, 2, 3, 4, 5, 6],'LeicRace': [0, 1],
 'hemda': [0, 1, 2], 'wstva':[],'LeicHBP':[0,1],'rYdiabe': [0, 1], 'raeducl':[2,3,1,0,4,np.nan], 'mstat':[
     3,1,5,7,4,8],'shlt':[2,3,1,4,5], 'hlthlm':[0,1,2],'hearte':[0,1], 'psyche':[0,1], 'physActive':[1,0],
  'smoken':[0,1,np.nan], 'jphysa':[1,3,2,5,4,np.nan], 'everHighGlu':[0,2,1], 'eatVegFru':[1,0],'mobilb':[],'lgmusa':[],
  'grossa':[], 'finea':[], 'drinkd_e':[],'itot':[],'cfoodo1m':[], 'estwt':[],'chol':[],'ldl':[], 'hdl':[],
  'trig':[], 'sys1':[], 'dias3':[], 'fglu':[], 'hba1c':[]
 }

path = 'NBayes.csv'
data = processing(labels=all_labels, to_replace=to_replace,all_labels=all_labels, values= values, path=path)
print(data.groupby(['rYdiabe']).describe())

        LeicGender                                              LeicRace  \
             count      mean       std  min  25%  50%  75%  max    count   
rYdiabe                                                                    
0           1687.0  0.452282  0.497865  0.0  0.0  0.0  1.0  1.0   1687.0   
1            322.0  0.540373  0.499143  0.0  0.0  1.0  1.0  1.0    322.0   

                   ... eatVegFru      everHighGlu                           \
             mean  ...       75%  max       count      mean       std  min   
rYdiabe            ...                                                       
0        0.016005  ...       1.0  1.0      1687.0  0.558388  0.789912  0.0   
1        0.034161  ...       1.0  1.0       322.0  1.186335  0.571438  0.0   

                             
         25%  50%  75%  max  
rYdiabe                      
0        0.0  0.0  1.0  2.0  
1        1.0  1.0  2.0  2.0  

[2 rows x 272 columns]
